In [ ]:
#default_exp ecoenv

## Google Colab preparations

In [ ]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

IN_MAIN = __name__ == '__main__'

In [ ]:
#Infrastructure for copying notebooks
if IN_COLAB and IN_MAIN:
    home_dir = '/content/drive/MyDrive/Colab Notebooks/Ecosystems/v3'
if IN_COLAB and IN_MAIN:
    from google.colab import drive
    drive.mount('/content/drive')
    import sys
    sys.path.append(home_dir)
    %cd $home_dir
    
!pip -q install import-ipynb

In [ ]:
#export
import gym
import numpy as np
import matplotlib.pyplot as plt
from gym import spaces
# import import_ipynb
from ecotwins.utility import distance, motion_diagram 
from ecotwins.animal_classes import Ecosystem, Terrain, Animal, SimpleSheep

importing Jupyter notebook from utility.ipynb
importing Jupyter notebook from animal_classes.ipynb
importing Jupyter notebook from perception.ipynb


In [ ]:
# Can we (re)move this cell?

ORIGIN = np.array([0.,0.])
SIDE = 20
N_OBJECTS = 10

# Agent settings
RADIUS = SIDE/2
DELTA = 0.01 #1e-2
REWARD_RADIUS = SIDE/10
# TRACE_LENGTH = 1000

# Training settings
EPISODE_LENGTH = 2000
# EPISODE_LENGTH = np.int(np.round(SIDE/DELTA))

In [ ]:
# Can we (re)move this cell?

def generate_objects(side=None, n_objects=N_OBJECTS):
    """Generates a random map of objects 
    param side: size of each side of the grid
    param n_objects: (initial) number of objects in the ecosystem
    """
    objects=(np.random.rand(n_objects,2)-0.5)*side # Takes values in (-ecoenv.SIDE/2,+ecoenv.SIDE/2)
    return objects

# The EcoEnv class

In [ ]:
#export
class EcoEnv(gym.Env): 
    """An ecosystem environment for OpenAI gym"""
    metadata = {'render.modes': ['human']} #Ta ej bort

    #A functon that takes an ecosystem and returns a gym
    def __init__(self, ecosystem):
        super(EcoEnv, self).__init__()
        
        self.ecosystem = ecosystem       
        self.agent = ecosystem.agent
        self.perception = ecosystem.agent.perception
        self.position = self.agent.position.copy()
        self.p_happiness = self.agent.happiness(self.ecosystem.terrain)
        self.current_step = 0
      
        self.action_space = self.agent.action_space
        self.observation_space = self.agent.observation_space
        # debug
        self.total_reward = 0
        # An attempt to reward stayin alive
        self.age_reward = (self.agent.happiness(self.ecosystem.terrain) /
                           self.agent.hyperparameters['max_age']
                          )

    def _next_observation(self):
        return self.agent.observation(self.ecosystem.terrain)

    # Helper function to step.
    def _take_action(self, action): 
        # Maybe this should be handled by the ecosystem ie the call should be
        # self.ecosystem.update(action, agent)
#         print(type(self))
        self.agent.update(action, self.ecosystem.terrain)
#     terrain update as well
        self.position = self.agent.position.copy()
        return self.position

    # Reward functions
    def _reward(self): 
        happiness = self.agent.happiness(self.ecosystem.terrain)
        r = happiness - self.p_happiness
        self.p_happiness = happiness
        # TODO: Clean handling of resource punishment
        return r + self.age_reward - 3 * self.agent.out_of_resources() 

    def _is_done(self):
        """Done if maximum number of step exceed or if
        we are outside the region. """ # or if we die?
        return self.current_step > EPISODE_LENGTH or self._is_outside()
    
    def _is_outside(self): return np.abs(self.position).max() > self.side / 2

    def _is_close(self):
        return distance(self.position, self.objects).min() < self.reward_radius

    # Execute one time step within the environment
    def step(self, action):
        self._take_action(action)
        self.current_step += 1
        
        reward = self._reward()
        self.total_reward += reward
        obs = self._next_observation()
        done = self.ecosystem.is_done()

        return obs, reward, done, {}

    # Reset the state of the environment to an initial state
    def reset(self):
        print(f"Reset@{self.current_step}, accumulated reward: {self.total_reward:.2f}", end="")
        print(", Interoception levels: ", end="")
        print(*[f'{k}:{v:.2f}' for k,v in self.agent.interoception.items()], sep=', ', end="")
        print(f' happiness: {self.agent.happiness(self.ecosystem.terrain):.2f}')
        self.total_reward = 0
        
        self.ecosystem.reset()
        self.position = self.agent.position.copy()
        self.current_step = 0
        self.p_happiness = self.agent.happiness(self.ecosystem.terrain)
#         self.consumed = []
        return self._next_observation()

    # Render the environment to the screen
    def render(self, trace, mode='human', close=False):
        # @TODO This should be handled properly
        side = self.ecosystem.terrain.space[0,1] - self.ecosystem.terrain.space[0,0]
        # @TODO Should support any number of object types not just one.
        objects = next(iter(self.ecosystem.terrain.objects.values()))
        motion_diagram(objects, trace, side)



In [ ]:
if IN_MAIN:
#     t = Terrain(objects={'dandelion':(np.random.random((10,2)) - 0.5) * 20})
    t = Terrain(objects={'dandelion': 10})
    hyperparameters = {'max_age': 2000, 'delta': 0.1, 'close': 1, 'gamma': 0.9}
    agent = SimpleSheep(distances={'dandelion':10}) 
    eco = Ecosystem(t, agent)
    env = EcoEnv(eco)

In [ ]:
from nbdev.export import notebook2script; notebook2script()

Converted 00_core.ipynb.
Converted animal_classes.ipynb.
Converted animation_helper.ipynb.
Converted ecoenv.ipynb.
Converted happiness.ipynb.
Converted index.ipynb.
Converted perception.ipynb.
Converted reflex_agent.ipynb.
Converted tyckande.ipynb.
Converted utility.ipynb.
Converted worlds.ipynb.
